## **Veritabanı Yedekleme ve Otomasyon Çalışması**

## **[https://www.kaggle.com/datasets/khushikyad001/covid-19-global-dataset](https://www.kaggle.com/datasets/khushikyad001/covid-19-global-dataset)**

### **1\. Yedekleme Süreçlerini Otomatikleştirme**

### **Veritabanı Log Tablosu Oluşturma**

In [1]:
USE [covid];
GO

CREATE TABLE BackupLog (
    id INT IDENTITY(1,1) PRIMARY KEY,
    database_name NVARCHAR(100),
    backup_start DATETIME,
    backup_end DATETIME,
    backup_size_MB FLOAT,
    backup_path NVARCHAR(500),
    error_message NVARCHAR(MAX),
    log_date DATETIME DEFAULT GETDATE()
);

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.159

### **SQL Server Agent Kontrolü**

In [2]:
SELECT servicename, status_desc
FROM sys.dm_server_services
WHERE servicename LIKE '%SQL Server Agent%';

(1 row affected)

Total execution time: 00:00:00.079

servicename,status_desc
SQL Server Agent (MSSQLSERVER),Running


### **Operatör ve Bildirim Ayarları**

In [9]:
USE msdb;
GO

EXEC dbo.sp_add_operator 
    @name = N'DBA_Team_2', 
    @email_address = N'persuren@gmail.com';
GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.020

In [10]:
USE msdb;
GO

EXEC dbo.sp_add_job @job_name = N'CovidDB_Backup_Job_New';
GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.052

In [11]:
USE msdb;
GO

EXEC dbo.sp_update_job 
    @job_name = N'CovidDB_Backup_Job_New',
    @notify_level_email = 2,
    @notify_email_operator_name = N'DBA_Team_2';
GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.029

### **Yedekleme İşini ve Adımlarını Oluştur**

In [12]:
USE msdb;
GO

EXEC sp_add_jobstep
    @job_name = N'CovidDB_Backup_Job_New',
    @step_name = N'Full_Backup',
    @subsystem = N'TSQL',
    @command = N'
        DECLARE @path NVARCHAR(500);
        SET @path = N''/var/opt/mssql/backups/covid_'' 
            + CONVERT(VARCHAR, GETDATE(), 112) + ''_'' 
            + REPLACE(CONVERT(VARCHAR, GETDATE(), 108), '':'', '''') + ''.bak'';
        BACKUP DATABASE [covid] TO DISK = @path WITH COMPRESSION, STATS = 10;
    ',
    @database_name = N'master';
GO


EXEC sp_add_jobstep
    @job_name = N'CovidDB_Backup_Job_New',
    @step_name = N'Cleanup_Old_Backups',
    @command = N'
        EXEC xp_delete_file 0, N''/var/opt/mssql/backups/'', N''bak'', DATEADD(DAY, -7, GETDATE());
    ',
    @database_name = N'master';
GO


EXEC sp_add_jobstep
    @job_name = N'CovidDB_Backup_Job_New',
    @step_name = N'Log_Backup_Info',
    @command = N'
        INSERT INTO covid.dbo.BackupLog (database_name, backup_start, backup_end, backup_size_MB, backup_path)
        SELECT 
            database_name, 
            backup_start_date, 
            backup_finish_date, 
            backup_size/1024/1024, 
            physical_device_name
        FROM msdb.dbo.backupset bs
        JOIN msdb.dbo.backupmediafamily bmf ON bs.media_set_id = bmf.media_set_id
        WHERE bs.database_name = ''covid''
          AND bs.type = ''D''
          AND backup_start_date >= DATEADD(HOUR, -1, GETDATE());
    ',
    @database_name = N'master';
GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.075

### **Zamanlama ve Aktivasyon**

In [13]:

EXEC dbo.sp_add_schedule
    @schedule_name = N'Daily_1AM',
    @freq_type = 4,
    @freq_interval = 1,
    @active_start_time = 010000;
GO


EXEC sp_attach_schedule
    @job_name = N'CovidDB_Backup_Job_New',
    @schedule_name = N'Daily_1AM';
GO


EXEC dbo.sp_add_jobserver
    @job_name = N'CovidDB_Backup_Job_New';
GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.143

### **2\. T-Sql ile Rapor Oluşturma**

In [2]:
USE covid;
GO

IF NOT EXISTS (SELECT * FROM sys.tables WHERE name = 'BackupLog')
BEGIN
    CREATE TABLE BackupLog (
        log_id INT IDENTITY(1,1) PRIMARY KEY,
        database_name NVARCHAR(255),
        error_message NVARCHAR(MAX),
        backup_date DATETIME DEFAULT GETDATE()
    );
END;
GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.039

### **Hata Yönetimi için Stored Procedure Güncellemesi**

In [3]:
USE covid;
GO

CREATE OR ALTER PROCEDURE usp_Backup_CovidDB
AS
BEGIN
    BEGIN TRY
        DECLARE @path NVARCHAR(500);
        SET @path = N'/var/opt/mssql/backups/covid_' 
            + CONVERT(VARCHAR, GETDATE(), 112) + '_' 
            + REPLACE(CONVERT(VARCHAR, GETDATE(), 108), ':', '') + '.bak';
        
        BACKUP DATABASE [covid] TO DISK = @path WITH COMPRESSION, STATS = 10;
    END TRY
    BEGIN CATCH
        INSERT INTO BackupLog (database_name, error_message)
        VALUES ('covid', ERROR_MESSAGE());
    END CATCH
END;

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.030

In [4]:
USE covid;
GO

SELECT * FROM BackupLog;
GO

Commands completed successfully.

(0 rows affected)

Total execution time: 00:00:00.019

id,database_name,backup_start,backup_end,backup_size_MB,backup_path,error_message,log_date


### **8\. Mail Adresi Oluşturma**

In [1]:
USE msdb;
GO
EXEC sp_configure 'show advanced options', 1;
RECONFIGURE;
EXEC sp_configure 'Database Mail XPs', 1;
RECONFIGURE;
GO

Commands completed successfully.

Configuration option 'show advanced options' changed from 1 to 1. Run the RECONFIGURE statement to install.

: Msg 15392, Level 16, State 1, Procedure sp_configure, Line 166
The specified option 'Database Mail XPs' is not supported by this edition of SQL Server and cannot be changed using sp_configure.

Total execution time: 00:00:00.078

## SMPT Hesap Ayarları

In [1]:
EXEC msdb.dbo.sysmail_add_account_sp
    @account_name = 'DBA_Notifications',
    @email_address = 'persuren@gmail.com',  
    @display_name = 'SQL Server DBA Alerts',
    @mailserver_name = 'smtp.gmail.com',  
    @port = 587,
    @username = 'persuren',      
    @password = 'A93dp3!',            
    @enable_ssl = 1;                        
GO

EXEC msdb.dbo.sysmail_add_profile_sp
    @profile_name = 'DBA_Profile';
GO

EXEC msdb.dbo.sysmail_add_profileaccount_sp
    @profile_name = 'DBA_Profile',
    @account_name = 'DBA_Notifications',
    @sequence_number = 1;
GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.120

In [2]:
USE msdb;
GO
EXEC dbo.sp_update_operator 
    @name = N'DBA_Team_2',
    @email_address = N'persuren@gmail.com', 
    @pager_address = N'', 
    @weekday_pager_start_time = 0, 
    @weekday_pager_end_time = 235959, 
    @pager_days = 0, 
    @netsend_address = N'', 
    @category_name = N'[Uncategorized]',
    @enabled = 1;
GO


Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.072

In [3]:
CREATE OR ALTER PROCEDURE usp_Backup_CovidDB
AS
BEGIN
    BEGIN TRY
        DECLARE @path NVARCHAR(500) = N'C:\Backup\covid.bak';
        BACKUP DATABASE [covid] TO DISK = @path WITH COMPRESSION;
    END TRY
    BEGIN CATCH
        DECLARE @errorMessage NVARCHAR(MAX) = ERROR_MESSAGE();

        INSERT INTO BackupLog (database_name, error_message) 
        VALUES ('covid', @errorMessage);

        EXEC msdb.dbo.sp_send_dbmail
            @profile_name = 'DBA_Profile',                    
            @recipients = 'persuren@gmail.com',               
            @subject = 'COVID Veritabanı Yedekleme Hatası',
            @body = 'Yedekleme işlemi sırasında şu hata oluştu: ' + @errorMessage;
    END CATCH
END;
GO

Commands completed successfully.

Total execution time: 00:00:00.088

In [10]:
USE covid;
GO

CREATE OR ALTER PROCEDURE usp_Backup_CovidDB
AS
BEGIN
    BEGIN TRY
        DECLARE @path NVARCHAR(500) = N'/invalid_directory/covid.bak';
        BACKUP DATABASE [covid] TO DISK = @path WITH COMPRESSION, STATS = 10;
    END TRY
    BEGIN CATCH
        INSERT INTO BackupLog (database_name, error_message)
        VALUES ('covid', ERROR_MESSAGE());
    END CATCH
END;
GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.039

In [11]:
USE covid;
GO
EXEC usp_Backup_CovidDB;

Commands completed successfully.

(1 row affected)

Total execution time: 00:00:00.120

In [12]:
USE covid;
GO
SELECT * FROM BackupLog;

Commands completed successfully.

(1 row affected)

Total execution time: 00:00:00.060

id,database_name,backup_start,backup_end,backup_size_MB,backup_path,error_message,log_date
1,covid,NULL,NULL,NULL,NULL,BACKUP DATABASE is terminating abnormally.,2025-05-04 15:02:38.360


In [13]:
BEGIN TRY
    DECLARE @path NVARCHAR(500) = N'/invalid_directory/covid.bak';
    BACKUP DATABASE [covid] TO DISK = @path;
END TRY
BEGIN CATCH
    SELECT ERROR_MESSAGE() AS ErrorMessage;
END CATCH

(1 row affected)

Total execution time: 00:00:00.207

ErrorMessage
BACKUP DATABASE is terminating abnormally.


In [14]:
USE covid;
GO
CREATE OR ALTER PROCEDURE usp_Backup_CovidDB
AS
BEGIN
    BEGIN TRY
        DECLARE @path NVARCHAR(500) = N'/invalid_path/covid.bak';
        BACKUP DATABASE [covid] TO DISK = @path WITH COMPRESSION;
    END TRY
    BEGIN CATCH
        DECLARE @errorMessage NVARCHAR(MAX) = ERROR_MESSAGE();
        INSERT INTO BackupLog (database_name, error_message) VALUES ('covid', @errorMessage);

        DECLARE @escapedErrorMessage NVARCHAR(MAX) = REPLACE(@errorMessage, '"', '\"');

        DECLARE @cmd NVARCHAR(MAX) = N'python3 /scripts/send_email.py "' + @escapedErrorMessage + '"';
        EXEC xp_cmdshell @cmd;
    END CATCH
END;
GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.054

In [15]:
USE covid;
GO
EXEC usp_Backup_CovidDB;

Commands completed successfully.

(1 row affected)

: Msg 15281, Level 16, State 1, Procedure xp_cmdshell, Line 1
SQL Server blocked access to procedure 'sys.xp_cmdshell' of component 'xp_cmdshell' because this component is turned off as part of the security configuration for this server. A system administrator can enable the use of 'xp_cmdshell' by using sp_configure. For more information about enabling 'xp_cmdshell', search for 'xp_cmdshell' in SQL Server Books Online.

Total execution time: 00:00:00.137

In [16]:
USE covid;
GO
SELECT * FROM BackupLog;

Commands completed successfully.

(2 rows affected)

Total execution time: 00:00:00.040

id,database_name,backup_start,backup_end,backup_size_MB,backup_path,error_message,log_date
1,covid,NULL,NULL,NULL,NULL,BACKUP DATABASE is terminating abnormally.,2025-05-04 15:02:38.360
2,covid,NULL,NULL,NULL,NULL,BACKUP DATABASE is terminating abnormally.,2025-05-04 15:18:27.420
